# Desafio

### Acesse a api da camara, e recupere os reembolsos do ano de 2018

In [1]:
import requests

# fazendo o download do conteúdo do arquivo
url = "http://www.camara.leg.br/cotas/Ano-2018.csv.zip"  # pega o arquivo .zip e na variavel r
r = requests.get(url)   

In [2]:
r.content
r.status_code

200

In [3]:
with open("Ano-2018.csv.zip", "wb") as arquivo:  # cria um novo arquivo para escrita em binario
    arquivo.write(r.content)                     # persiste o conteudo de r neste arquivo .zip


In [4]:
from zipfile import ZipFile
# descompacto todo o conteudo de .zip para nova variavel
zipado = ZipFile("Ano-2018.csv.zip", "r")
# listo os arquivos descompactados   
zipado.namelist()                           

['Ano-2018.csv']

In [5]:
# extraio um dos arquivos para nova pasta
zipado.extract(member="Ano-2018.csv", path="reembolso-2018") 

'reembolso-2018\\Ano-2018.csv'

In [6]:
zipado.close()

### Leia o Arquivo csv 

In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)
DTYPE = {
    'txNomeParlamentar': str,
    'cpf' : str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str
}

df_ano_2018 = pd.read_csv("reembolso-2018/Ano-2018.csv", sep=';', dtype=DTYPE)
df_ano_2018.shape

(292632, 31)

### Visualize 10 linhas aleatórias

In [9]:
df_ano_2018.sample(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
101173,Eduardo da Fonte,53467108415,141421,156,2015,PE,PP,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - GOL,075.756.510/0015-9,Bilhete: QQMQRE,0,2018-03-22T00:00:00,996.93,0,996.93,3,2018,0,Eduardo da Fonte,BSB/REC,0,0,NaN,1822,1544529,NaN


### Existem registros com `ideDocumento` nulo?

In [15]:
# df_ano_2018.info()
filtro = df_ano_2018.ideDocumento.isna()
df_ano_2018[filtro]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento


### Converta a coluna `datEmissao` para `datetime`, e descubra quantos reembolsos aconteram no primeiro semestre

In [16]:
df_ano_2018["datEmissao"] = pd.to_datetime(
    df_ano_2018.datEmissao,
    format="%Y-%m-%d"
)

In [17]:
df_ano_2018.head(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,BISCOITOS CASEIROS HOMONNAI LTDA,046.448.200/0017-7,10473,4,2018-12-20,454.41,0,454.41,12,2018,0,NaN,NaN,1550817,NaN,NaN,2812,6728796,http://camara.leg.br/cota-parlamentar/nota-fis...


In [23]:
# df_ano_2018.set_index("datEmissao").loc["2018-01-01":"2018-04-30"]
df_ano_2018["datEmissao"].dt.dayofweek >= 5

0         False
1         False
2         False
3         False
4         False
          ...  
292627    False
292628    False
292629    False
292630    False
292631    False
Name: datEmissao, Length: 292632, dtype: bool

In [26]:
df_ano_2018 = df_ano_2018.fillna(value='Não se aplica')

In [27]:
df_ano_2018.head(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,Não se aplica,Não se aplica,Não se aplica,2015,Não se aplica,Não se aplica,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,Não se aplica,BISCOITOS CASEIROS HOMONNAI LTDA,046.448.200/0017-7,10473,4,2018-12-20 00:00:00,454.41,0,454.41,12,2018,0,Não se aplica,Não se aplica,1550817,Não se aplica,Não se aplica,2812,6728796,http://camara.leg.br/cota-parlamentar/nota-fis...


### Limpe a coluna `txtCNPJCPF`: ela deve conter somente dígitos numéricos

In [28]:
# vamos trocar qualquer caracter que não seja um dígito número por '', essa operação remove os dígitos
df_ano_2018["txtCNPJCPF"] = df_ano_2018["txtCNPJCPF"].str.replace(r'\D', '', regex=True)

df_ano_2018.txtCNPJCPF

0         04644820000177
1         10638597000158
2         24882567000105
3         24882567000105
4         03713903000108
               ...      
292627    00000000000006
292628    00982933000121
292629    27402097000115
292630    02214932000162
292631    02012862000160
Name: txtCNPJCPF, Length: 292632, dtype: object

### Quais são os valores únicos do campo `indTipoDocumento`? Substitua o conteúdo pela respectiva legenda

*Dica*: 0 (Zero), para Nota Fiscal; 1 (um), para Recibo; e 2, para Despesa no Exterior.

In [29]:
df_ano_2018.indTipoDocumento.unique()


array(['4', '0', '1', '2', '3'], dtype=object)

In [79]:
df_ano_2018.indTipoDocumento.value_counts()


0    177181
4     63975
1     51313
3       130
2        33
Name: indTipoDocumento, dtype: int64

In [36]:
df_ano_2018.txtDescricao.value_counts(normalize=True).sort_values(ascending=False).to_frame().head()

,txtDescricao
PASSAGEM AÉREA - RPA,0.315157
COMBUSTÍVEIS E LUBRIFICANTES.,0.211399
TELEFONIA,0.105358
"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0.081228
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,0.073492


In [42]:
df_ano_2018.vlrLiquido.describe()

count    292632.000000
mean        736.164516
std        2481.479508
min       -4906.000000
25%          50.000000
50%         192.450000
75%         671.730000
max      130000.000000
Name: vlrLiquido, dtype: float64

In [44]:
filtro_valor_max = df_ano_2018.vlrLiquido == df_ano_2018.vlrLiquido.max()

df_ano_2018[filtro_valor_max]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
67059,GIVALDO CARIMBÃO,10270060510,74558,166,2015,AL,Não se aplica,55,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,0,Não se aplica,GRAFMARQUES INDUSTRIA EDITORA E SERVICOS LTDA,00887925000104,1645,4,2018-12-21 00:00:00,130000.0,0,130000.0,12,2018,0,Não se aplica,Não se aplica,1554078,Não se aplica,Não se aplica,1332,6737727,http://camara.leg.br/cota-parlamentar/nota-fis...


Visualizar o gastos por partido 

In [61]:
df_ano_2018.groupby(['sgPartido', 'sgUF', 'txNomeParlamentar'])['vlrLiquido'].sum().sort_values(ascending=False).to_frame()

vlrLiquido
sgPartido     sgUF          txNomeParlamentar                  
Não se aplica RR            Jhonatan de Jesus         561401.20
                            REMÍDIO MONAI             549060.53
              AC            Jéssica Sales             547381.07
              SP            Arlindo Chinaglia         545289.58
              RR            CARLOS ANDRADE            544175.67
...                                                         ...
              Não se aplica LIDERANÇA DO DEMOCRATAS    14707.35
                            LID.GOV-CD                 12065.54
                            LID.GOV-CN                  3682.12
                            LIDERANÇA DO AVANTE         1781.02
                            LIDERANÇA DO PHS             920.83

[568 rows x 1 columns]

In [66]:
pd.options.display.float_format = '{:,.2f}'.format

df_ano_2018.groupby(['txtCNPJCPF', 'txtFornecedor'])['vlrLiquido'].sum().sort_values(ascending=False).to_frame()[:10]

,,vlrLiquido
txtCNPJCPF,txtFornecedor,
07575651000159,Cia Aérea - GOL,"21,161,930.40"
02012862000160,Cia Aérea - TAM,"19,199,593.85"
09296295000160,Cia Aérea - AZUL,"6,705,916.83"
02575829000148,Cia Aérea - AVIANCA,"5,684,992.80"
00000000000001,CELULAR FUNCIONAL,"1,468,850.29"
02558157000162,TELEFONICA BRASIL S.A.,"1,217,088.87"
05443449000148,VERTICE - SOCIEDADE CIVIL DE PROFISSIONAIS ASSOCIADOS,"987,100.00"
07976147002295,Movida Locação de Veículos LTDA,"776,340.61"
23721017000133,"T2 COMUNICACAO, VIDEO E PRODUCOES EIRELI - EPP","698,933.34"
